In [ ]:
from vista_ssm import vista_ssm_Funcs as vista
import numpy as np
import random
import sklearn.datasets
from scipy.stats import ortho_group

In [ ]:
#data generation
param_dic={'DIM_X': 5,
           'DIM_Y': 2,
           'NUM_DATA': 120,
           'N_CLUSTER' : 3,
           'NUM_CPU' : 30,
           'FIX' : [],
           'NUM_LGSSM' : 30,
           'MAX_ITER' : 1000,
           'EPSILON' : 0.1,
           'BIC' : True}
#generating random time steps
T=40
T_diff=25
tp=[]
for n in range(param_dic['NUM_DATA']):
    individual_T=random.randint(T-T_diff,T+T_diff)
    individual_points=[0]
    for t in range(1,individual_T):
        individual_points.append(individual_points[-1]+random.randint(1,5))
    tp.append(np.array(individual_points)/individual_points[-1])
tp=np.array(tp,dtype=object)

#setting parameters
params=vista.initializationmethod('random',param_dic,None,None) #initializing MLGSSM with random parameters

params['Gamma'][0] = 0.001*np.eye(param_dic['DIM_X'])
params['Gamma'][1] = 0.001*np.eye(param_dic['DIM_X'])
params['Gamma'][2] = 0.002*np.eye(param_dic['DIM_X'])

params['Sigma'][0] = 0.001*np.eye(param_dic['DIM_Y'])
params['Sigma'][1] = 0.002*np.eye(param_dic['DIM_Y'])
params['Sigma'][2] = 0.001*np.eye(param_dic['DIM_Y'])

params['P'][0] = 0.005*np.eye(param_dic['DIM_X'])
params['P'][1] = 0.005*np.eye(param_dic['DIM_X'])
params['P'][2] = 0.005*np.eye(param_dic['DIM_X'])

params['mu'][0] = np.array([0.3, 0., 0.4, -0.1, -0.1], ndmin=2).transpose()
params['mu'][1] = np.array([0., -0.2, 0.1, -0.3, 0.6], ndmin=2).transpose()
params['mu'][2] = np.array([0.1, 0.3, 0.1, -0.2, 0.], ndmin=2).transpose()

params['C'][0] =np.array([[1., 1., 1., 1.,1], [1., 0., 0., 0., 1]])
params['C'][1] =np.array([[1., 1., 1., 1.,1], [0., 1., 1., 0., 0.]])
params['C'][2] =np.array([[1., 1., 1., 1.,1.], [1., 1., 0., 0., 1.]])

O1=ortho_group.rvs(dim=param_dic['DIM_X'])
D1=4*np.diag(np.array([-0.5,0.2,-0.3,0.,0.]))
params['A'][0] = np.matmul(np.matmul(O1,D1),O1.transpose())
params['A'][1] = np.array([[0.4, 0., 0., 0., 0.],[0., 0., -3., 0., 0.],[0.,3., 0., 0., 0.],[0.,0.,0.,0, 4],[0., 0., 0.,-4, 0]])
O3=np.eye(param_dic['DIM_X'])
D3=np.array([[0., 5., 0., 0., 0.],[-5., 0., -0., 0., 0.],[0.,0., 0.5, 0., 0.],[0.,0.,0.,-1., 0.],[0., 0., 0.,0., 0.5]])
params['A'][2] = -np.matmul(np.matmul(O3,D3),O3.transpose())

(data,tp,label)=vista.mlgssmSample([40,50,30],params,T,True,T_diff,time_points=tp) #generating MLGSSM

In [ ]:
simul={'param':params,
 'data': data,
 'tp':tp,
 'label':label}
vista.savedic(simul,'../data/sim_data.pickle')